# Demo Coroutines

A process-oriented DES often has a concept of passing control from one "thread" to another.

<img src="./media/Threads-Concept-01.png" width=500>

In the general case, this idea of a thread may actually be better described as a "*coroutine*". Python's `asyncio` library can mode coroutines.

### A Space Misison Coroutine

Consider some individual vehicle in our simulation. Lets call it *Alpha*.

In [1]:
from dataclasses import dataclass

@dataclass
class Vehicle:
    name: str

vehicle = Vehicle("Alpha")

This kind of vehicle performs an activity called *ascent* -- which is the process of flying that starts with **liftoff** and ends with **stage**.

In [2]:
import asyncio

@dataclass
class SpaceEvent:
    name: str
    sync: asyncio.Event = asyncio.Event()

liftoff = SpaceEvent("liftoff")
stage   = SpaceEvent("stage")

liftoff

SpaceEvent(name='liftoff', sync=<asyncio.locks.Event object at 0x1065b1a50 [unset]>)

In [3]:
async def ascent(event_start, event_complete, vehicle): 
    print(f"Event {event_start.name} nas occurred")
    print(f"{vehicle.name}: Start Ascent")

    await event_complete.wait()

    print(f"{vehicle.name}: Complete Ascent")

In [4]:
@dataclass
class Simulation:
    future: list  # of events
    active: list  # of tasks
    complete: list  # of tasks
    
    def schedule(self, activity, event_start, event_complete, vehicle):

        new_task = asyncio.create_task(activity(event_start, event_complete, vehicle))
        # update the sim state
        self.future.append(event_complete)
        self.active.append(new_task)

        new_task.add_done_callback(self.update_state)

        # self.complete(new_task)

    def update_state(self, task):

        # self.future.remove(event)
        self.active.remove(task)
        self.complete.append(task)


In [5]:
sim = Simulation([], [], [])

vehicle = Vehicle("Alpha")

liftoff = SpaceEvent("liftoff")
stage   = SpaceEvent("stage")

sim.schedule(ascent, liftoff, stage, vehicle)

# sim.active
sim

Simulation(future=[SpaceEvent(name='stage', sync=<asyncio.locks.Event object at 0x1065b1a50 [unset]>)], active=[<Task pending name='Task-4' coro=<ascent() running at /var/folders/d4/714vzybx28lcn2xx4n__4n8m0000gp/T/ipykernel_89736/1073127561.py:1> cb=[Simulation.update_state()]>], complete=[])

In [10]:
sim.active

[]

Task exception was never retrieved
future: <Task finished name='Task-4' coro=<ascent() done, defined at /var/folders/d4/714vzybx28lcn2xx4n__4n8m0000gp/T/ipykernel_89736/1073127561.py:1> exception=AttributeError("'SpaceEvent' object has no attribute 'wait'")>
Traceback (most recent call last):
  File "/var/folders/d4/714vzybx28lcn2xx4n__4n8m0000gp/T/ipykernel_89736/1073127561.py", line 5, in ascent
    await event_complete.wait()
AttributeError: 'SpaceEvent' object has no attribute 'wait'


In [7]:
stage.sync.set()

In [8]:
sim.active

[]

In [9]:
sim.complete

[<Task finished name='Task-4' coro=<ascent() done, defined at /var/folders/d4/714vzybx28lcn2xx4n__4n8m0000gp/T/ipykernel_89736/1073127561.py:1> exception=AttributeError("'SpaceEvent' object has no attribute 'wait'")>]